<a href="https://colab.research.google.com/github/josephthomaa/ML-Notebooks/blob/main/Llama_finetuned_quantized_JT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash

git clone https://github.com/PromtEngineer/localGPT.git
ls

localGPT
sample_data


Cloning into 'localGPT'...


In [51]:


pip install -r localGPT/requirements.txt


In [52]:
%%bash



# rm -r localGPT/DB/
python localGPT/ingest.py

In [5]:
import os
import logging
import click
import torch
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler  # for streaming response
from langchain.callbacks.manager import CallbackManager

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

from localGPT.prompt_template_utils import get_prompt_template

# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.vectorstores import Chroma
from transformers import (
    GenerationConfig,
    pipeline,
)

# from localGPT.load_models import (
#     load_quantized_model_gguf_ggml,
#     load_quantized_model_qptq,
#     load_full_model,
# )

# from localGPT.constants import (
#     EMBEDDING_MODEL_NAME,
#     PERSIST_DIRECTORY,
#     MODEL_ID,
#     MODEL_BASENAME,
#     MAX_NEW_TOKENS,
#     MODELS_PATH,
# )
from pprint import pprint

In [ ]:
# Finetuned quantized model

# MODEL_ID = "ggml-model-q4_k_m_new.gguf"
# MODEL_BASENAME = "ggml-model-q4_k_m_new.gguf"

In [6]:
# Pretrained quantized model

MODEL_ID = "TheBloke/Llama-2-7b-Chat-GGUF"
MODEL_BASENAME = "llama-2-7b-chat.Q4_K_M.gguf"
EMBEDDING_MODEL_NAME = "hkunlp/instructor-large"

import os

# from dotenv import load_dotenv
from chromadb.config import Settings

# https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/excel.html?highlight=xlsx#microsoft-excel
from langchain.document_loaders import CSVLoader, PDFMinerLoader, TextLoader, UnstructuredExcelLoader, Docx2txtLoader
from langchain.document_loaders import UnstructuredFileLoader, UnstructuredMarkdownLoader


# load_dotenv()
ROOT_DIRECTORY = "localGPT"

# Define the folder for storing database
SOURCE_DIRECTORY = f"{ROOT_DIRECTORY}/SOURCE_DOCUMENTS"

PERSIST_DIRECTORY = f"{ROOT_DIRECTORY}/DB"

MODELS_PATH = "./models"

# Can be changed to a specific number
INGEST_THREADS = os.cpu_count() or 8

# Define the Chroma settings
CHROMA_SETTINGS = Settings(
    anonymized_telemetry=False,
    is_persistent=True,
)

# Context Window and Max New Tokens
CONTEXT_WINDOW_SIZE = 4096
MAX_NEW_TOKENS = CONTEXT_WINDOW_SIZE  # int(CONTEXT_WINDOW_SIZE/4)

#### If you get a "not enough space in the buffer" error, you should reduce the values below, start with half of the original values and keep halving the value until the error stops appearing

N_GPU_LAYERS = 100  # Llama-2-70B has 83 layers
N_BATCH = 512

In [7]:
embeddings = HuggingFaceInstructEmbeddings(model_name=EMBEDDING_MODEL_NAME, model_kwargs={"device": "cpu"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [8]:
db = Chroma(
        persist_directory=PERSIST_DIRECTORY,
        embedding_function=embeddings,
    )
retriever = db.as_retriever()

In [9]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

use_history = False
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
system_prompt = """You are a helpful assistant, you will use the provided context to answer user questions.
Read the given context before answering questions and think step by step. If you can not answer a user question based on
the provided context, inform the user. Do not use any other information for answering user. Provide a detailed answer to the question."""


# system_prompt = """You are a helpful assistant, you will use the provided context to answer user questions.
# Read the given context before answering questions.
# If it contains a sequence of instructions, re-write those instructions in the following format:

# Step 1 - ...
# Step 2 - …
# …
# Step N - …

# If the text does not contain a sequence of instructions, then only write the summary of the text.
# If you can not answer a user question based on the provided context, inform the user.
# Do not use any other information for answering user. Provide a detailed answer to the question."""


# prompt, memory = get_prompt_template(promptTemplate_type="llama", history=use_history)


In [10]:
instruction = """
            Context: {context}
            User: {question}"""
SYSTEM_PROMPT = B_SYS + system_prompt + E_SYS
prompt_template = B_INST + SYSTEM_PROMPT + instruction + E_INST
prompt = PromptTemplate(input_variables=["context", "question"], template=prompt_template)
memory = ConversationBufferMemory(input_key="question", memory_key="history")

In [11]:
import torch
from auto_gptq import AutoGPTQForCausalLM
from huggingface_hub import hf_hub_download
from langchain.llms import LlamaCpp

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    LlamaForCausalLM,
    LlamaTokenizer,
)

In [12]:
model_path = hf_hub_download(
            repo_id=MODEL_ID,
            filename=MODEL_BASENAME,
            resume_download=True,
            cache_dir=MODELS_PATH,
        )


In [13]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 35.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.11-cp310-cp310-manylinux_2_35_x86_64.whl size=1023486 sha256=b4568484b16501e68438bd33063aa054f9fd2be22462b1389b846963ce1209a5
  Stored in directory: /root/.cache/pip/wheels/dc/42/77/a3ab0d02700427ea364de5797786c0272779dce795f62c3bc2
Successfully built llama-cpp-python


In [33]:
kwargs = {
            "model_path": model_path,
            "n_ctx": CONTEXT_WINDOW_SIZE,
            "max_tokens": MAX_NEW_TOKENS,
            "n_batch": N_BATCH,  # set this based on your GPU & CPU RAM
        }
llm = LlamaCpp(**kwargs)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [15]:
qa = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",  # try other chains types as well. refine, map_reduce, map_rerank
            retriever=retriever,
            return_source_documents=True,  # verbose=True,
            callbacks=callback_manager,
            chain_type_kwargs={
                "prompt": prompt,
            },
        )

In [16]:
# quantized model finetuned
query = "What steps to do when there is a order trend low scenerio ?"
res = qa(query)
answer, docs = res["result"], res["source_documents"]
print(answer)

  Great, thank you for your question! In the context of the Standard Operating Procedure you provided, when there is an order trend low scenario, the following steps should be taken:
1. Confirm Low Order Volume: Verify that there is indeed a decrease in order volume compared to previous periods. (Step 1 of the procedure)
2. Review Historical Data: Analyze historical order data to identify any patterns, seasonality, or anomalies that might explain the decrease. (Step 2 of the procedure)
3. Examine Production Services: Review the status of critical microservices and components responsible for order processing, payment, and inventory management. (Step 3 of the procedure)
4. Performance Metrics: Analyze CPU usage, memory consumption, network activity, and response times for each service. (Step 4 of the procedure)
5. Issue Mitigation: Implement corrective actions based on validated hypotheses. (Step 6 of the procedure)
6. Document Findings: Maintain detailed records of findings, actions tak

In [47]:
from typing import Sequence, Optional
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.llms import OpenAI
from pydantic import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser


class Task(BaseModel):
    task_name: str

class Tasks(BaseModel):
    """Identifying information about all tasks in a text."""
    people: Sequence[Task]


parser = PydanticOutputParser(pydantic_object=Tasks)

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"


instruction = """\
For the following context, extract the following information:

task_name: Extract any automation task name that can be used for the user query and output them as json in following format.
json format ```
{{{{
    "task_name": string \ name of the automation task
}}}}
context: {context}
User: {question}
"""
# SYSTEM_PROMPT = B_SYS + system_prompt + E_SYS
# prompt_template = B_INST + SYSTEM_PROMPT + instruction + E_INST
prompt_template = B_INST + instruction + E_INST
prompt = PromptTemplate(input_variables=["context", "question"], template=prompt_template)
memory = ConversationBufferMemory(input_key="question", memory_key="history")

In [48]:
qa = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",  # try other chains types as well. refine, map_reduce, map_rerank
            retriever=retriever,
            return_source_documents=True,  # verbose=True,
            callbacks=callback_manager,
            chain_type_kwargs={
                "prompt": prompt,
            },
        )

In [49]:
query = "Which automation task should we run for low order trend scenerio"
res = qa(query)
answer, docs = res["result"], res["source_documents"]
print(answer)

Llama.generate: prefix-match hit


  Based on the provided context, the following information can be extracted:
Task Name:
* Low Order Sanity AO
* Validate External API Response Time
* Load Balancing and Scaling
* Security and DDoS

Context:
* OPS Automation Tasks
* Objective: Different automation tasks that the ops team can use.

Please find the extracted information in the format of a JSON object below:
{{
    "task_name": [
        "Low Order Sanity AO",
        "Validate External API Response Time",
        "Load Balancing and Scaling",
        "Security and DDoS"

    ]
}}


In [28]:
retrieved_docs = retriever.invoke("Which automation task should we run for low order trend scenerio")
print(retrieved_docs[0].page_content)

OPS Automation Tasks :

Objective: Different automation tasks ops team can use.

Low Order Sanity AO:

		Name: Low Order Sanity AO

		Description: This task involves running an automation pipeline to check if both EIP (Enterprise Integration Patterns) and FRP (Functional Requirement Parameters) flows are passed for a low order trend scenario.

		Repository: The automation pipeline for this task is configured in the 'APP1-ops-synthetic-monitoring' repository.

		Schedule: The schedule for this automation pipeline is configured within the 'APP1-ops-synthetic-monitoring' repository.

		Validate External API Response Time:

		Name: Validate External API Response Time

		Description: This task involves running an automation pipeline to check if the response time or failure rate is above a threshold value for a low order trend scenario, likely involving external API calls.

		Repository: The automation pipeline for this task is configured in the 'APP1-ops-validator-pipelines' repository.


In [35]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain

# Schema
schema = {
    "properties": {
        "name": {"type": "string"},
    },
    "required": ["name"],
}

# Input
inp = retrieved_docs[0].page_content

# Run chain
chain = create_extraction_chain(schema, llm)
chain.run(inp)

TypeError: ignored

In [50]:
parser.parse(answer)

OutputParserException: ignored

In [53]:
!pip install llama-index

In [42]:
from llama_index.program import (
    BasePydanticProgram,
    DFFullProgram,
    DataFrame,
    DataFrameRowsOnly,
)

program = BasePydanticProgram(
    output_cls=DataFrame,
    llm=llm,
    prompt_template_str=(
        "Please extract the following query into a structured data according"
        " to: {input_str}.Please extract both the set of column names and a"
        " set of rows."
    ),
    verbose=True,
)

response_obj = program(
    input_str="""My name is John and I am 25 years old. I live in
        New York and I like to play basketball. His name is
        Mike and he is 30 years old. He lives in San Francisco
        and he likes to play baseball. Sarah is 20 years old
        and she lives in Los Angeles. She likes to play tennis.
        Her name is Mary and she is 35 years old.
        She lives in Chicago."""
)
response_obj

TypeError: ignored

In [ ]:
query = "Find any automation that can be used for query : 'What are the steps for low order trend scenerio'"
res = qa(query)
answer, docs = res["result"], res["source_documents"]
print(answer)

Llama.generate: prefix-match hit


  Based on the provided context and user query, the following automation tasks can be identified as relevant for the low order trend scenario:
JSON Output:
{
"task_name": "Low Order Sanity AO",
"value": "APP1-ops-synthetic-monitoring"
}
{
"task_name": "Validate External API Response time",
"value": "APP1-ops-validator-pipelines"
}
These tasks are identified based on the provided user query, which mentions the scenario of low order trend. The automation tasks listed above are relevant for this scenario, as they are designed to monitor and validate the performance of external APIs and flows in the APP1 environment. By running these tasks, the OPS team can identify any issues or anomalies that may be impacting the order trend and take corrective action accordingly.



llama_print_timings:        load time = 58653.81 ms
llama_print_timings:      sample time =   223.70 ms /   194 runs   (    1.15 ms per token,   867.23 tokens per second)
llama_print_timings: prompt eval time =  7368.94 ms /    60 tokens (  122.82 ms per token,     8.14 tokens per second)
llama_print_timings:        eval time = 31826.27 ms /   193 runs   (  164.90 ms per token,     6.06 tokens per second)
llama_print_timings:       total time = 40084.63 ms


In [ ]:
query = "Find any automation that can be used for query : 'What are the steps for low order trend scenerio'"
res = qa(query)
answer, docs = res["result"], res["source_documents"]
print(answer)

  Based on the provided context, the following are the task names that can be used for the user query:
* "Low order Sanity AO"
* "Validate External API Response time"

The output in JSON format is:
{
"task_name": ["Low order Sanity AO", "Validate External API Response time"]
}



llama_print_timings:        load time = 58601.99 ms
llama_print_timings:      sample time =    90.51 ms /    80 runs   (    1.13 ms per token,   883.85 tokens per second)
llama_print_timings: prompt eval time = 117849.35 ms /  1000 tokens (  117.85 ms per token,     8.49 tokens per second)
llama_print_timings:        eval time = 13006.12 ms /    79 runs   (  164.63 ms per token,     6.07 tokens per second)
llama_print_timings:       total time = 131204.30 ms


In [ ]:
from pydantic import BaseModel
from typing import Optional
from llama_cpp import LogitsProcessorList
from lmformatenforcer import CharacterLevelParser, JsonSchemaParser
from lmformatenforcer.integrations.llamacpp import build_llamacpp_logits_processor

DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\
"""

instruction = """\
For the following context and user query, extract the following information:

context: {context}
User: {question}

task_name: Extract any automation task name that can be used for the user query and output them as a comma separated Python list.

You MUST answer using the following json schema:
task_name : str

"""


def get_prompt(message: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f'<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{message} [/INST]'

class AnswerFormat(BaseModel):
    task_name: str

# question = 'Please give me information about Michael Jordan. You MUST answer using the following json schema: '
# question_with_schema = f'{question}{AnswerFormat.schema_json()}'
schema = AnswerFormat.schema_json()
prompt = get_prompt(question_with_schema)
prompt_template = B_INST + instruction + E_INST
prompt = PromptTemplate(input_variables=["context", "question"], template=prompt_template)

logits_processors = LogitsProcessorList([build_llamacpp_logits_processor(llm._model, JsonSchemaParser(AnswerFormat.schema()))])

qa = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",  # try other chains types as well. refine, map_reduce, map_rerank
            retriever=retriever,
            return_source_documents=True,  # verbose=True,
            callbacks=callback_manager,
            chain_type_kwargs={
                "prompt": prompt,
                "logits_processor": logits_processors,
            },
        )

AttributeError: 'LlamaCpp' object has no attribute '_model'

In [ ]:
output_template = """\
For the following text, extract the following information:

task_name: Extract any automation task name that can be used for the user query and output them as a comma separated Python list.

Format the output as JSON with the following keys:
task_name

text: {text}
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(output_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ntask_name: Extract any automation task name that can be used for the user query and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ntask_name\n\ntext: {text}\n'))]
